# Assignment 3a: Basic Gradio RAG Frontend
## Day 6 Session 2 - Building Simple RAG Applications

In this assignment, you'll build a simple Gradio frontend for your RAG system with just the essential features:
- Button to initialize the vector database
- Search query input and button
- Display of AI responses

**Learning Objectives:**
- Create basic Gradio interfaces
- Connect RAG backend to frontend
- Handle user interactions and database initialization
- Build functional AI-powered web applications

**Prerequisites:**
- Completed Assignment 1 (Vector Database Basics)
- Completed Assignment 2 (Advanced RAG)
- Understanding of LlamaIndex fundamentals


## 📚 Part 1: Setup and Imports

Import all necessary libraries for building your Gradio RAG application.


In [7]:
%pip install -q -r '../requirements.txt'

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: "'../requirements.txt'"


In [1]:
# Import required libraries
import gradio as gr
import os
from pathlib import Path

# LlamaIndex components
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, Settings
from llama_index.vector_stores.lancedb import LanceDBVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openrouter import OpenRouter

print("✅ All libraries imported successfully!")


c:\Users\jeya_\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ All libraries imported successfully!


## 🤖 Part 2: RAG Backend Class

Create a simple RAG backend that can initialize the database and answer queries.


In [7]:
import os
os.environ["OPENROUTER_API_KEY"] = "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"

In [8]:
class SimpleRAGBackend:
    """Simple RAG backend for Gradio frontend."""
    
    def __init__(self):
        self.index = None
        self.setup_settings()
    
    def setup_settings(self):
        """Configure LlamaIndex settings."""
        # Set up the LLM using OpenRouter
        api_key = os.getenv("OPENROUTER_API_KEY")
        if api_key:
            Settings.llm = OpenRouter(
                api_key=api_key,
                model="gpt-4o",
                temperature=0.1
            )
        
        # Set up the embedding model
        Settings.embed_model = HuggingFaceEmbedding(
            model_name="BAAI/bge-small-en-v1.5",
            trust_remote_code=True
        )
        
        # Set chunking parameters
        Settings.chunk_size = 512
        Settings.chunk_overlap = 50
    
    def initialize_database(self, data_folder="../data"):
        """Initialize the vector database with documents."""
        # Check if data folder exists
        if not Path(data_folder).exists():
            return f"❌ Data folder '{data_folder}' not found!"
        
        try:
            # Create vector store
            vector_store = LanceDBVectorStore(
                uri="./basic_rag_vectordb",
                table_name="documents"
            )
            
            # Load documents
            reader = SimpleDirectoryReader(input_dir=data_folder, recursive=True)
            documents = reader.load_data()
            
            # Create storage context and index
            storage_context = StorageContext.from_defaults(vector_store=vector_store)
            self.index = VectorStoreIndex.from_documents(
                documents, 
                storage_context=storage_context,
                show_progress=True
            )
            
            return f"✅ Database initialized successfully with {len(documents)} documents!"
        
        except Exception as e:
            return f"❌ Error initializing database: {str(e)}"
    
    def query(self, question):
        """Query the RAG system and return response."""
        # Check if index exists
        if self.index is None:
            return "❌ Please initialize the database first!"
        
        # Check if question is empty
        if not question or not question.strip():
            return "⚠️ Please enter a question first!"
        
        try:
            # Create query engine and get response
            query_engine = self.index.as_query_engine()
            response = query_engine.query(question)
            return str(response)
        
        except Exception as e:
            return f"❌ Error processing query: {str(e)}"

# Initialize the backend
rag_backend = SimpleRAGBackend()
print("🚀 RAG Backend initialized and ready!")


2026-02-08 14:33:28,327 - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5


2026-02-08 14:33:28,621 - INFO - HTTP Request: HEAD https://huggingface.co/BAAI/bge-small-en-v1.5/resolve/main/modules.json "HTTP/1.1 307 Temporary Redirect"
2026-02-08 14:33:28,641 - INFO - HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/BAAI/bge-small-en-v1.5/5c38ec7c405ec4b44b94cc5a9bb96e735b38267a/modules.json "HTTP/1.1 200 OK"
2026-02-08 14:33:28,875 - INFO - HTTP Request: HEAD https://huggingface.co/BAAI/bge-small-en-v1.5/resolve/main/config_sentence_transformers.json "HTTP/1.1 307 Temporary Redirect"
2026-02-08 14:33:28,891 - INFO - HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/BAAI/bge-small-en-v1.5/5c38ec7c405ec4b44b94cc5a9bb96e735b38267a/config_sentence_transformers.json "HTTP/1.1 200 OK"
2026-02-08 14:33:29,136 - INFO - HTTP Request: HEAD https://huggingface.co/BAAI/bge-small-en-v1.5/resolve/main/config_sentence_transformers.json "HTTP/1.1 307 Temporary Redirect"
2026-02-08 14:33:29,140 - WARNING - Warning: You are sending unauthentica

🚀 RAG Backend initialized and ready!


## 🎨 Part 3: Gradio Interface

Create a simple Gradio interface with:
1. Button to initialize the database
2. Text input for queries
3. Button to submit queries
4. Text output for responses
5. Text output for status messages


In [9]:
def create_basic_rag_interface():
    """Create basic RAG interface with essential features."""
    
    def initialize_db():
        """Handle database initialization."""
        return rag_backend.initialize_database()
    
    def handle_query(question):
        """Handle user queries."""
        return rag_backend.query(question)
    
    # Create Gradio interface using gr.Blocks()
    with gr.Blocks(title="Basic RAG Assistant") as interface:
        # Add title and description
        gr.Markdown("# 🤖 Basic RAG Assistant")
        gr.Markdown("Initialize the database and ask questions about your documents!")
        
        # Add initialization section
        gr.Markdown("## Step 1: Initialize Database")
        init_btn = gr.Button("Initialize Database", variant="primary", size="lg")
        status_output = gr.Textbox(
            label="Status",
            interactive=False,
            lines=2
        )
        
        # Add query section  
        gr.Markdown("## Step 2: Ask Questions")
        query_input = gr.Textbox(
            label="Your Question",
            placeholder="Ask a question about the documents...",
            lines=3
        )
        
        submit_btn = gr.Button("Ask Question", variant="primary", size="lg")
        
        response_output = gr.Textbox(
            label="Response",
            interactive=False,
            lines=6
        )
        
        # Connect buttons to functions
        init_btn.click(initialize_db, outputs=[status_output])
        submit_btn.click(handle_query, inputs=[query_input], outputs=[response_output])
        
    return interface

# Create the interface
basic_interface = create_basic_rag_interface()
print("✅ Basic RAG interface created successfully!")

2026-02-08 14:33:47,493 - INFO - HTTP Request: HEAD https://huggingface.co/api/telemetry/https%3A/api.gradio.app/gradio-initiated-analytics "HTTP/1.1 200 OK"


✅ Basic RAG interface created successfully!


2026-02-08 14:33:48,578 - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


## 🚀 Part 4: Launch Your Application

Launch your Gradio application and test it!


In [ ]:
print("🎉 Launching your Basic RAG Assistant...")
print("🔗 Your application will open in a new browser tab!")
print("")
print("📋 Testing Instructions:")
print("1. Click 'Initialize Database' button first")
print("2. Wait for success message")
print("3. Enter a question in the query box")
print("4. Click 'Ask Question' to get AI response")
print("")
print("💡 Example questions to try:")
print("- What are the main topics in the documents?")
print("- Summarize the key findings")
print("- Explain the methodology used")
print("")
print("🚀 Launch your app:")

# Your launch code here:
# Uncomment when implemented
basic_interface.launch()

🎉 Launching your Basic RAG Assistant...
🔗 Your application will open in a new browser tab!

📋 Testing Instructions:
1. Click 'Initialize Database' button first
2. Wait for success message
3. Enter a question in the query box
4. Click 'Ask Question' to get AI response

💡 Example questions to try:
- What are the main topics in the documents?
- Summarize the key findings
- Explain the methodology used

🚀 Launch your app:
* Running on local URL:  http://127.0.0.1:7861


2026-02-08 14:34:02,107 - INFO - HTTP Request: GET http://127.0.0.1:7861/gradio_api/startup-events "HTTP/1.1 200 OK"
2026-02-08 14:34:02,153 - INFO - HTTP Request: HEAD http://127.0.0.1:7861/ "HTTP/1.1 200 OK"


* To create a public link, set `share=True` in `launch()`.


2026-02-08 14:34:03,143 - INFO - HTTP Request: HEAD https://huggingface.co/api/telemetry/https%3A/api.gradio.app/gradio-launched-telemetry "HTTP/1.1 200 OK"


Failed to load file c:\Users\jeya_\Documents\AIMasterMind\MyWorkspace\Day7_RAG\ai-accelerator-C3\Day_7\session_2\assignments\..\data\audio\ai_agents.mp3 with error: [WinError 2] The system cannot find the file specified. Skipping...
Failed to load file c:\Users\jeya_\Documents\AIMasterMind\MyWorkspace\Day7_RAG\ai-accelerator-C3\Day_7\session_2\assignments\..\data\audio\in_the_end.mp3 with error: [WinError 2] The system cannot find the file specified. Skipping...
Failed to load file c:\Users\jeya_\Documents\AIMasterMind\MyWorkspace\Day7_RAG\ai-accelerator-C3\Day_7\session_2\assignments\..\data\audio\rags.mp3 with error: [WinError 2] The system cannot find the file specified. Skipping...


c:\Users\jeya_\AppData\Local\Programs\Python\Python313\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Generating embeddings:  32%|███▏      | 30/94 [00:45<01:37,  1.52s/it]

Failed to load file c:\Users\jeya_\Documents\AIMasterMind\MyWorkspace\Day7_RAG\ai-accelerator-C3\Day_7\session_2\assignments\..\data\audio\ai_agents.mp3 with error: [WinError 2] The system cannot find the file specified. Skipping...


c:\Users\jeya_\AppData\Local\Programs\Python\Python313\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Failed to load file c:\Users\jeya_\Documents\AIMasterMind\MyWorkspace\Day7_RAG\ai-accelerator-C3\Day_7\session_2\assignments\..\data\audio\in_the_end.mp3 with error: [WinError 2] The system cannot find the file specified. Skipping...
Failed to load file c:\Users\jeya_\Documents\AIMasterMind\MyWorkspace\Day7_RAG\ai-accelerator-C3\Day_7\session_2\assignments\..\data\audio\rags.mp3 with error: [WinError 2] The system cannot find the file specified. Skipping...


Generating embeddings: 100%|██████████| 94/94 [03:35<00:00,  2.30s/it]
2026-02-08 14:40:33,243 - INFO - query_type :, vector
2026-02-08 14:40:41,181 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


In [11]:
# Stop the application running on local URL
print("🛑 Stopping the Basic RAG Assistant...")
print("Closing the Gradio server and all connections...")

try:
    # Close the Gradio interface
    basic_interface.close()
    print("✅ Gradio server stopped successfully")
    print("🌐 Local URL application has been terminated")
except Exception as e:
    print(f"⚠️ Error closing interface: {str(e)}")

print("✨ Application cleanup complete!")

🛑 Stopping the Basic RAG Assistant...
Closing the Gradio server and all connections...
Closing server running on port: 7861
✅ Gradio server stopped successfully
🌐 Local URL application has been terminated
✨ Application cleanup complete!


## ✅ Assignment Completion Checklist

Before submitting, ensure you have:

- [x] RAG backend is provided and working
- [ ] Created Gradio interface with required components:
  - [ ] Title and description using gr.Markdown()
  - [ ] Initialize database button using gr.Button()
  - [ ] Status output using gr.Textbox()
  - [ ] Query input field using gr.Textbox()
  - [ ] Submit query button using gr.Button()
  - [ ] Response output area using gr.Textbox()
- [ ] Connected buttons to backend functions using .click()
- [ ] Successfully launched the application
- [ ] Tested the full workflow (initialize → query → response)

## 🎊 Congratulations!

You've successfully built your first Gradio RAG application! You now have:

- A functional web interface for your RAG system
- Understanding of Gradio basics and component connections
- A foundation for building more complex AI applications

**Next Steps**: Complete Assignment 3b to add advanced configuration options to your RAG interface!
